# BERT를 이용한 텍스트 분류 Simple Guide
(이글은 https://medium.com/swlh/a-simple-guide-on-using-bert-for-text-classification-bbf041ac8d04 을 참조하였음을 알려드립니다.)  


우리는 지금부터 google의 bert를 통해 텍스트 이진분류를 알아보겠다,  
이글은 bert모델을 어떻게 파인튜닝하고 이것을 어떻게 텍스트 분류에 이용할지 최대한 간단하고 직관적이게 설명하는 것을 목표로 한다.    

## 1. Intro

시작하기 전에 , https://github.com/ThilinaRajapakse/BERT_binary_text_classification
우리의 예제 코드는 이곳에 있다.  
모든 코드 가이드는 이곳에서 설명토록 하겠다.  
편한하게 참조하고 repository 들을 복제 하여 가이드를 따라 공부하세요.  

만약 당신이 BERT 에 대해 처음이라면 아래 논문을 읽어주세요. 이곳에선 BERT의 세부적인 테크닉은 알려주지 않습니다.   
https://arxiv.org/pdf/1810.04805.pdf   
그리고 만약 당신이 Transformer modle 이나('attention', 'embedding', 그리고 'encoder-decoder'에 익숙지 못하다면 아래 블로그의 글을 참조하기 원합니다.  
[아래글은 제 github에도 정리가 되어있습니다.]
http://jalammar.github.io/illustrated-transformer/
물론 이글을 이해하는데 BERT나 Transformer의 모든것을 알 필요는 없습니다.  
그러나 위의 글들은 당신이 BERT나 Transformer를 좀더 이해하는데 쉽게 해 줄 것입니다.  
지금까진 우리가 하지 않을것에 대해 알아보았고, 그럼 앞으로 우리가 해야될것을 알아보도록 하겠습니다.  

- Getting BERT downloaded and set up. We will be using the PyTorch version provided by the amazing folks at Hugging Face.
- Converting a dataset in the .csv format to the .tsv format that BERT knows and loves.
- Loading the .tsv files into a notebook and converting the text representations to a feature representation (think numerical) that the BERT model can work with.
- Setting up a pretrained BERT model for fine-tuning.
- Fine-tuning a BERT model.
- Evaluating the performance of the BERT model.

마지막으로 들어가기전에 , 우리는 jupyter notebook 을 통해 데이터를 정제하고, 학습시키고 그리고 eavaluation 할것 입니다. 

# 2. Getting set up

BERT 를 구현해보자 .   
1. 가상환경에 필요한 package들을 설치하겠습니다. 어떠한 pakage 나 evironment manager를 사용하던 상관 없습니다 . 여기서는 Conda를 이용하였습니다.  

- conda create -n bert python pytorch pandas tqdm
- conda install -c anaconda scikit-learn

아나콘다 기본 사용법 : https://niceman.tistory.com/85

2. Install the PyTorch version of BERT from Hugging Face.
    - pip install pytorch-pretrained-bert


3. text classification 을 하기 위해서,  우리는 textclassification datasets 이 필요합니다. 이곳 가이드 에선,  우리는 Yelp Reviews Polarity datasets 를 사용할 것 입니다. 이곳 fast.ai 에서 데이타셋을 받을수 있습니다, https://s3.amazonaws.com/fast-ai-nlp/yelp_review_polarity_csv.tgz 다운을 받은뒤 압축을 풀어 train.csv 파일과 test.csv파일들을 아래의 주소와 같이 저장해 주세요. 
    - [starting_directory]/data/train.csv



# 3 Preparing data

본격적으로 요리를 시작하기 전에 우리는 제료들이 필요합니다. 
대부분의 우리가 찾는 datasets 들은 전형적으로 csv 파일형태입니다. 이곳에서 쓰일 Yelp Review datasest 또한 예외는 아닙니다.   
Pandas로 이 datasets 를 열어 확인해보도록 하겠습니다.  

In [1]:
import pandas as pd

In [2]:
train_df = pd.read_csv('data/train.csv', header=None)
train_df.head()

,0,1
0,1,"Unfortunately, the frustration of being Dr. Go..."
1,2,Been going to Dr. Goldberg for over 10 years. ...
2,1,I don't know what Dr. Goldberg was like before...
3,1,I'm writing this review to give you a heads up...
4,2,All the food is great here. But the best thing...


In [3]:
test_df = pd.read_csv('data/test.csv', header=None)
test_df.head()

,0,1
0,2,"Contrary to other reviews, I have zero complai..."
1,1,Last summer I had an appointment to get new ti...
2,2,"Friendly staff, same starbucks fair you get an..."
3,1,The food is good. Unfortunately the service is...
4,2,Even when we didn't have a car Filene's Baseme...


위에서 보듯, 위 데이터들은 train.csv 파일과 test.csv 파일 두개의 csv 파일로 구성되어 있습니다 . 이들은 header 도 없고, 라벨과 text 를 위한 두개의 columns 를 가지고 있습니다. 이 라벨은 이곳에서 조금 이상하게 쓰입니다. 다른 라벨들과는 다르게 (일반적으로 0,1 을 사용 하여 구별) 이곳에서는 1과 를 통해 라벨을 구별합니다.
    - 1은 나쁘다는 리뷰
    - 2는 좋다는 리뷰
우리는 그래서 이것을 좀더 익숙하게 쓰기 위해 0,1라벨링으로 바꾸도록 하겠습니다.
    - 0은 나쁘다는 리뷰
    - 1은 좋다는 리뷰

In [4]:
train_df[0] = (train_df[0] == 2).astype(int)
test_df[0] = (test_df[0] == 2).astype(int)

In [5]:
train_df.head()

,0,1
0,0,"Unfortunately, the frustration of being Dr. Go..."
1,1,Been going to Dr. Goldberg for over 10 years. ...
2,0,I don't know what Dr. Goldberg was like before...
3,0,I'm writing this review to give you a heads up...
4,1,All the food is great here. But the best thing...


In [6]:
test_df.head()

,0,1
0,1,"Contrary to other reviews, I have zero complai..."
1,0,Last summer I had an appointment to get new ti...
2,1,"Friendly staff, same starbucks fair you get an..."
3,0,The food is good. Unfortunately the service is...
4,1,Even when we didn't have a car Filene's Baseme...


훨씬 좋아 진것 같죠??? 

그러나 BERT는 데이터가 아래와 같이 특정 형식(헤더 행이 없는 4개의 열)의 tsv 파일에 있기를 원합니다. 
    - Column 0: An ID for the row
    - Column 1: The label for the row (should be an int)
    - Column 2: A column of the same letter for all rows. BERT wants this so we’ll give it, but we don’t have a use for it.
    - Column 3: The text for the row
(솔직히 왜 이렇게 쓰는지 잘 모르겠습니다... )

In [7]:
train_df_bert = pd.DataFrame({
    'id':range(len(train_df)),
    'label':train_df[0],
    'alpha':['a']*train_df.shape[0],
    'text': train_df[1].replace(r'\n', ' ', regex=True)
})

train_df_bert.head()

,id,label,alpha,text
0,0,0,a,"Unfortunately, the frustration of being Dr. Go..."
1,1,1,a,Been going to Dr. Goldberg for over 10 years. ...
2,2,0,a,I don't know what Dr. Goldberg was like before...
3,3,0,a,I'm writing this review to give you a heads up...
4,4,1,a,All the food is great here. But the best thing...


In [8]:
dev_df_bert = pd.DataFrame({
    'id':range(len(test_df)),
    'label':test_df[0],
    'alpha':['a']*test_df.shape[0],
    'text': test_df[1].replace(r'\n', ' ', regex=True)
})

dev_df_bert.head()

,id,label,alpha,text
0,0,1,a,"Contrary to other reviews, I have zero complai..."
1,1,0,a,Last summer I had an appointment to get new ti...
2,2,1,a,"Friendly staff, same starbucks fair you get an..."
3,3,0,a,The food is good. Unfortunately the service is...
4,4,1,a,Even when we didn't have a car Filene's Baseme...


편의를 위해, 앞으로 test data를 dev data라 명하겠습니다.  
우리는 train data를 우리 모델을 학습시키기 위해 사용하고 ,  
dev data는 퍼포먼스를 평가하기 위해 사용하도록 하겠습니다.  
BERET의 데이터 로딩 클레스도 test 파일을 사용할 수 도 있지만 test 파일이 unlabellded 될 수도 있기때문에 , 우리는 train 파일과 dev 파일을 대신하여 사용합니다.  ( 뭔 씨소리여 )  

그러면 이제 정확한 형태의 데이터를 얻었으니 우리는 이제 train, dev 파일을 tsv 파일로 저장하면 됩니다.  

In [9]:
train_df_bert.to_csv('data/train.tsv', sep='\t', index=False, header=False)

In [10]:
dev_df_bert.to_csv('data/dev.tsv', sep='\t', index=False, header=False)

이제 제료가 준비 된겁니다 .그럼 본격적으로 BERT 를 요리해 보겠습니다. 

# Data to Features
fine-tuning 을 전 마지막단계는 데이터를 BERT가 사용할 수 있도록 전환하는 것입니다. 
코드의 대부분은 HuggingFace 예제 run_classifier.py에서 적용되었습니다.  

그럼 이제부터 우리가 이전 섹션에서 데이터를 .tsv 형태로 변경한 이유에 대해 알아보도록 하겠습니다.  
그것은 우리가 이진 분류 작업을 위해 BERT와 함께 제공되는 예제 클래스를 쉽게 재사용할 수 있게 해줍니다.  
여기 그 클래스 들이 있습니다. 

In [11]:
from __future__ import absolute_import, division, print_function

import csv
import os
import sys
import logging

logger = logging.getLogger()
csv.field_size_limit(2147483647) # Increase CSV reader's field limit incase we have long text.


class InputExample(object):
    """A single training/test example for simple sequence classification."""

    def __init__(self, guid, text_a, text_b=None, label=None):
        """Constructs a InputExample.
        Args:
            guid: Unique id for the example.
            text_a: string. The untokenized text of the first sequence. For single
            sequence tasks, only this sequence must be specified.
            text_b: (Optional) string. The untokenized text of the second sequence.
            Only must be specified for sequence pair tasks.
            label: (Optional) string. The label of the example. This should be
            specified for train and dev examples, but not for test examples.
        """
        self.guid = guid
        self.text_a = text_a
        self.text_b = text_b
        self.label = label


class DataProcessor(object):
    """Base class for data converters for sequence classification data sets."""

    def get_train_examples(self, data_dir):
        """Gets a collection of `InputExample`s for the train set."""
        raise NotImplementedError()

    def get_dev_examples(self, data_dir):
        """Gets a collection of `InputExample`s for the dev set."""
        raise NotImplementedError()

    def get_labels(self):
        """Gets the list of labels for this data set."""
        raise NotImplementedError()

    @classmethod
    def _read_tsv(cls, input_file, quotechar=None):
        """Reads a tab separated value file."""
        with open(input_file, "r", encoding="utf-8") as f:
            reader = csv.reader(f, delimiter="\t", quotechar=quotechar)
            lines = []
            for line in reader:
                if sys.version_info[0] == 2:
                    line = list(unicode(cell, 'utf-8') for cell in line)
                lines.append(line)
            return lines


class BinaryClassificationProcessor(DataProcessor):
    """Processor for binary classification dataset."""

    def get_train_examples(self, data_dir):
        """See base class."""
        return self._create_examples(
            self._read_tsv(os.path.join(data_dir, "train.tsv")), "train")

    def get_dev_examples(self, data_dir):
        """See base class."""
        return self._create_examples(
            self._read_tsv(os.path.join(data_dir, "dev.tsv")), "dev")

    def get_labels(self):
        """See base class."""
        return ["0", "1"]

    def _create_examples(self, lines, set_type):
        """Creates examples for the training and dev sets."""
        examples = []
        for (i, line) in enumerate(lines):
            guid = "%s-%s" % (set_type, i)
            text_a = line[3]
            label = line[1]
            examples.append(
                InputExample(guid=guid, text_a=text_a, text_b=None, label=label))
        return examples

### InputExample
그럼 각 클래스에 대해 알아보도록 하겠습니다.  
먼저 첫번째 클래스로 InputExample가 있습니다. 첫 번째 클래스인 InputExample은 우리의 datasets의 단일예제가 포함되어야 하는 형태입니다. text_b 속성은 이진 분류 작업에 필요하지 않으므로 사용하지 않을 것입니다. 다른 속성은 상당히 자명합니다.   

### DataProcessor, BinaryClassificationProcessor
다른 두번째 클래스는 DataProcessor 와 BinaryClassificationProcessor 입니다. 
이것들은 tsv.file 들을 읽고 실제 BERT 모델에 궁국적으로 들어갈 형태로 전환시켜주는 것을 준비하는데 도움을 주는 클래스들입니다.  

### BinaryClassificationProcessor
BinaryClassificationProcessor 클래스는 train.tsv 및 dev.tsv 파일을 읽고 InputExample 객체 목록으로 변환할 수 있습니다.
(객체목록으로 변환할 수 있다는게 무슨 말일까..?)
(Python 실력이 턱없이 부족하다 ㅠ)

BERT는 토큰화 이 후 시퀀스 킬이에 대한 제한이 있습니다. 대부분의 BERT 모델의 경우 Tokenization 이후의 시퀀스 길이는 512 입니다.  그러나 속도를 위하여 우리는 여기서 이보다 더 작거나 동일하게 세팅을 하도록 하겠습니다.  
저는 시퀀스 길이의 최대치를 128로 하였습니다.  
물론 이것보다 좀더 긴 모델의 경우 더 좋은 결과를 만들어 낼 수 있습니다.  

### InputFeature 
InputFeature는 BERT모델에 들어갈 형태의 순수한 숫자 데이터로 구성되어 있습니다. 
이는 각 예제의 텍스트를 토큰화하고 더 긴 시퀀스를 자르는 동시에 주어진 최대 시퀀스 길이(128)로 더 짧은 시퀀스를 채우는 방식으로 준비됩니다.(어떻게 padding이 되는가?)
  
저는 InputExample 객체를 InputFeature 객체로 변환하는 작업이 기본적으로 상당히 느리다는 것을 알게 되었고, Python의 'multiprocessing' 라이브러리를 활용하도록 변환 코드를 수정하여 공정을 크게 가속화할 수 있었습니다..

In [12]:
class InputFeatures(object):
    """A single set of features of data."""

    def __init__(self, input_ids, input_mask, segment_ids, label_id):
        self.input_ids = input_ids
        self.input_mask = input_mask
        self.segment_ids = segment_ids
        self.label_id = label_id


def _truncate_seq_pair(tokens_a, tokens_b, max_length):
    """Truncates a sequence pair in place to the maximum length."""

    # This is a simple heuristic which will always truncate the longer sequence
    # one token at a time. This makes more sense than truncating an equal percent
    # of tokens from each, since if one sequence is very short then each token
    # that's truncated likely contains more information than a longer sequence.
    while True:
        total_length = len(tokens_a) + len(tokens_b)
        if total_length <= max_length:
            break
        if len(tokens_a) > len(tokens_b):
            tokens_a.pop()
        else:
            tokens_b.pop()


def convert_example_to_feature(example_row):
    # return example_row
    example, label_map, max_seq_length, tokenizer, output_mode = example_row

    tokens_a = tokenizer.tokenize(example.text_a)

    tokens_b = None
    if example.text_b:
        tokens_b = tokenizer.tokenize(example.text_b)
        # Modifies `tokens_a` and `tokens_b` in place so that the total
        # length is less than the specified length.
        # Account for [CLS], [SEP], [SEP] with "- 3"
        _truncate_seq_pair(tokens_a, tokens_b, max_seq_length - 3)
    else:
        # Account for [CLS] and [SEP] with "- 2"
        if len(tokens_a) > max_seq_length - 2:
            tokens_a = tokens_a[:(max_seq_length - 2)]

    tokens = ["[CLS]"] + tokens_a + ["[SEP]"]
    segment_ids = [0] * len(tokens)

    if tokens_b:
        tokens += tokens_b + ["[SEP]"]
        segment_ids += [1] * (len(tokens_b) + 1)

    input_ids = tokenizer.convert_tokens_to_ids(tokens)

    # The mask has 1 for real tokens and 0 for padding tokens. Only real
    # tokens are attended to.
    input_mask = [1] * len(input_ids)

    # Zero-pad up to the sequence length.
    padding = [0] * (max_seq_length - len(input_ids))
    input_ids += padding
    input_mask += padding
    segment_ids += padding

    assert len(input_ids) == max_seq_length
    assert len(input_mask) == max_seq_length
    assert len(segment_ids) == max_seq_length

    if output_mode == "classification":
        label_id = label_map[example.label]
    elif output_mode == "regression":
        label_id = float(example.label)
    else:
        raise KeyError(output_mode)

    return InputFeatures(input_ids=input_ids,
                         input_mask=input_mask,
                         segment_ids=segment_ids,
                         label_id=label_id)

우리는 이것들이 어떻게 쓰이는지는 이후에 보도록 하겠습니다.

(  
그리고 이 source 는 convert_examples_to_features.py 로 따로 저장하세요 그리고 
5번째 라인의 null 을 None 으로 수정하세요 :)  
)

자 이제 시작으로 우리가 필요로 하는 packages 들을 import 해보겠습니다. 

In [44]:
import torch
import pickle
from torch.utils.data import (DataLoader, RandomSampler, SequentialSampler, TensorDataset)
from torch.nn import CrossEntropyLoss, MSELoss

from tqdm import tqdm_notebook, trange
import os
from pytorch_pretrained_bert import BertTokenizer, BertModel, BertForMaskedLM, BertForSequenceClassification
from pytorch_pretrained_bert.optimization import BertAdam, WarmupLinearSchedule

from multiprocessing import Pool, cpu_count
from tools import *
import convert_examples_to_features

# OPTIONAL: if you want to have more information on what's happening, activate the logger as follows
import logging
logging.basicConfig(level=logging.INFO)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [45]:
# The input data dir. Should contain the .tsv files (or other data files) for the task.
DATA_DIR = "data/"

# Bert pre-trained model selected in the list: bert-base-uncased, 
# bert-large-uncased, bert-base-cased, bert-large-cased, bert-base-multilingual-uncased,
# bert-base-multilingual-cased, bert-base-chinese.
BERT_MODEL = 'bert-base-cased'

# The name of the task to train.I'm going to name this 'yelp'.
TASK_NAME = 'yelp'

# The output directory where the fine-tuned model and checkpoints will be written.
OUTPUT_DIR = f'outputs/{TASK_NAME}/'

# The directory where the evaluation reports will be written to.
REPORTS_DIR = f'reports/{TASK_NAME}_evaluation_report/'

# This is where BERT will look for pre-trained models to load parameters from.
CACHE_DIR = 'cache/'

# The maximum total input sequence length after WordPiece tokenization.
# Sequences longer than this will be truncated, and sequences shorter than this will be padded.
MAX_SEQ_LENGTH = 128

TRAIN_BATCH_SIZE = 24
EVAL_BATCH_SIZE = 32
LEARNING_RATE = 2e-5
NUM_TRAIN_EPOCHS = 1
RANDOM_SEED = 42
GRADIENT_ACCUMULATION_STEPS = 1
WARMUP_PROPORTION = 0.1
OUTPUT_MODE = 'classification'

CONFIG_NAME = "config.json"
WEIGHTS_NAME = "pytorch_model.bin"

In [46]:
output_mode = OUTPUT_MODE

cache_dir = CACHE_DIR

In [47]:
if os.path.exists(REPORTS_DIR) and os.listdir(REPORTS_DIR):
        REPORTS_DIR += f'/report_{len(os.listdir(REPORTS_DIR))}'
        os.makedirs(REPORTS_DIR)
if not os.path.exists(REPORTS_DIR):
    os.makedirs(REPORTS_DIR)
    REPORTS_DIR += f'/report_{len(os.listdir(REPORTS_DIR))}'
    os.makedirs(REPORTS_DIR)

In [48]:
if os.path.exists(OUTPUT_DIR) and os.listdir(OUTPUT_DIR):
        raise ValueError("Output directory ({}) already exists and is not empty.".format(OUTPUT_DIR))
if not os.path.exists(OUTPUT_DIR):
    os.makedirs(OUTPUT_DIR)

In the first cell, we are importing the necessary packages.  
In the next cell, we are setting some paths for where files should be stored and where certain files can be found.   
We are also setting some configuration options for the BERT model.   
Finally, we will create the directories if they do not already exist.  


Next, we will use our BinaryClassificationProcessor to load in the data, and get everything ready for the tokenization step.

In [49]:
processor = BinaryClassificationProcessor()
train_examples = processor.get_train_examples(DATA_DIR)
train_examples_len = len(train_examples)
print(train_examples_len)

560000


In [50]:
label_list = processor.get_labels() # [0, 1] for binary classification
num_labels = len(label_list)

In [51]:
num_train_optimization_steps = int(
    train_examples_len / TRAIN_BATCH_SIZE / GRADIENT_ACCUMULATION_STEPS) * NUM_TRAIN_EPOCHS

In [52]:
# Load pre-trained model tokenizer (vocabulary)
tokenizer = BertTokenizer.from_pretrained('bert-base-cased', do_lower_case=False)

INFO:pytorch_pretrained_bert.tokenization:loading vocabulary file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-cased-vocab.txt from cache at C:\Users\ashgh\.pytorch_pretrained_bert\5e8a2b4893d13790ed4150ca1906be5f7a03d6c4ddf62296c383f6db42814db2.e13dbb970cb325137104fb2e5f36fe865f27746c6b526f6352861b1980eb80b1


In [53]:
label_map = {label: i for i, label in enumerate(label_list)}
train_examples_for_processing = [(example, label_map, MAX_SEQ_LENGTH, tokenizer, OUTPUT_MODE) for example in train_examples]

이제 우리는 우리만의 BinaryClassificationProcessor 만들었습니다. 그리고 이것을 train examples을 로드 하는데 이용하였습니다.  
그럼 이제 모델을 트레이닝하는 동안에 쓸 변수들을 세팅해보도록 하겠습니다.  
그다음 BERT로 pretrained 된 tokenizer 를 불러오도록 하겠습니다.  
이경우,  우리는 bert-base-cased model을 이용할 것입니다.  

convert_example_to_feature 함수는 예제, 라벨 맵, 최대 시퀀스 길이, 토큰라이저 및 출력 모드를 포함하는 튜플을 예상합니다.  
따라서 마지막으로 convert_example_to_feature 함수에 의해 처리될 수 있는 예제 목록(토큰화, 잘라내기/붙여넣기, InputFeatures로 변환)을 만들 도록 하겠습니다.

Now, we can use the multi-core goodness of modern CPU’s to process the examples (relatively) quickly. My Ryzen 7 2700x took about one and a half hours for this part.

In [54]:
process_count = cpu_count() - 1
if __name__ ==  '__main__':
    print(f'Preparing to convert {train_examples_len} examples..')
    print(f'Spawning {process_count} processes..')
    with Pool(process_count) as p:
        train_features = list(tqdm_notebook(p.imap(convert_examples_to_features.convert_example_to_feature, train_examples_for_processing), total=train_examples_len))

Preparing to convert 560000 examples..
Spawning 11 processes..


AttributeError: module 'convert_examples_to_features' has no attribute 'convert_example_to_feature'

In [55]:
with open(DATA_DIR + "train_features.pkl", "wb") as f:
    pickle.dump(train_features, f)

NameError: name 'train_features' is not defined

In [57]:
'''
If you are having trouble multiprocessing inside Notebooks, give this script a shot.
'''

import torch
from tqdm import tqdm
from pytorch_pretrained_bert import BertTokenizer


from multiprocessing import Pool, cpu_count
import pickle
from tools import *
import convert_examples_to_features

# OPTIONAL: if you want to have more information on what's happening, activate the logger as follows
import logging
logging.basicConfig(level=logging.INFO)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# The input data dir. Should contain the .tsv files (or other data files) for the task.
DATA_DIR = "data/"

# Bert pre-trained model selected in the list: bert-base-uncased,
# bert-large-uncased, bert-base-cased, bert-large-cased, bert-base-multilingual-uncased,
# bert-base-multilingual-cased, bert-base-chinese.
BERT_MODEL = 'bert-base-cased'

# The name of the task to train.
TASK_NAME = 'yelp'

# The output directory where the model predictions and checkpoints will be written.
OUTPUT_DIR = f'outputs/{TASK_NAME}/'

CACHE_DIR = 'cache/'

# The maximum total input sequence length after WordPiece tokenization.
# Sequences longer than this will be truncated, and sequences shorter than this will be padded.
MAX_SEQ_LENGTH = 128

TRAIN_BATCH_SIZE = 24
EVAL_BATCH_SIZE = 8
LEARNING_RATE = 2e-5
NUM_TRAIN_EPOCHS = 1
RANDOM_SEED = 42
GRADIENT_ACCUMULATION_STEPS = 1
WARMUP_PROPORTION = 0.1
OUTPUT_MODE = 'classification'

CONFIG_NAME = "config.json"
WEIGHTS_NAME = "pytorch_model.bin"

output_mode = OUTPUT_MODE

# Load pre-trained model tokenizer (vocabulary)
tokenizer = BertTokenizer.from_pretrained('bert-base-cased', do_lower_case=False)

processor = BinaryClassificationProcessor()
train_examples = processor.get_train_examples(DATA_DIR)
train_examples_len = len(train_examples)

label_list = processor.get_labels() # [0, 1] for binary classification
num_labels = len(label_list)

label_map = {label: i for i, label in enumerate(label_list)}
train_examples_for_processing = [(example, label_map, MAX_SEQ_LENGTH, tokenizer, output_mode) for example in train_examples]

process_count = cpu_count() - 2
# Running time on Ryzen 7 2700x with these settings is about 1 hour
if __name__ == '__main__':
    print(f'Preparing to convert {train_examples_len} examples..')
    print(f'Spawning {process_count} processes..')
    with Pool(process_count) as p:
        train_features = list(tqdm(p.imap(convert_examples_to_features.convert_example_to_feature, train_examples_for_processing), total=train_examples_len))

with open("train_features.pkl", 'wb') as f:
    pickle.dump(train_features, f)

INFO:pytorch_pretrained_bert.tokenization:loading vocabulary file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-cased-vocab.txt from cache at C:\Users\ashgh\.pytorch_pretrained_bert\5e8a2b4893d13790ed4150ca1906be5f7a03d6c4ddf62296c383f6db42814db2.e13dbb970cb325137104fb2e5f36fe865f27746c6b526f6352861b1980eb80b1


Preparing to convert 560000 examples..
Spawning 10 processes..


AttributeError: module 'convert_examples_to_features' has no attribute 'convert_example_to_feature'

# Fine-tuning BERT (finally!)


Had your coffee? Raring to go? Let’s show BERT how it’s done! (Fine tune. Show how it’s done. Get it? I might be bad at puns.)
Not much left now, let’s hope for smooth sailing. (Or smooth.. cooking? I forgot my analogy somewhere along the way. Anyway, we now have all the ingredients in the pot, and all we have to do is turn on the stove and let thermodynamics work its magic.)

In [ ]:

# Load pre-trained model (weights)
model = BertForSequenceClassification.from_pretrained(BERT_MODEL, cache_dir=CACHE_DIR, num_labels=num_labels)
# model = BertForSequenceClassification.from_pretrained(CACHE_DIR + 'cased_base_bert_pytorch.tar.gz', cache_dir=CACHE_DIR, num_labels=num_labels)

INFO:pytorch_pretrained_bert.file_utils:https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-cased.tar.gz not found in cache, downloading to C:\Users\ashgh\AppData\Local\Temp\tmp03czucc2
  2%|█                                                                  | 6736896/404400730 [00:43<21:57, 301858.30B/s]

In [ ]:
model.to(device)

여기서부턴 다음시간에 그리고 위에 예제 해결 할 방법 찾아보자 